In [1]:
from shiny import App, render, ui, reactive
import pandas as pd
# import faicons


excel_file = pd.ExcelFile("stock.xlsx")

df_inv = pd.read_excel(excel_file, sheet_name="inventory")
df_check = pd.read_excel(excel_file, sheet_name="checkouts")
person_dict = pd.read_excel(excel_file, sheet_name="person_dict")
dept_dict = pd.read_excel(excel_file, sheet_name="dept_dict")

person_list = person_dict["full_name"].unique().tolist()

item_dict = df_inv.set_index("item_id")["item_name"].to_dict()
item_list = df_inv["item_name"].unique().tolist()

nested_dict = {}
for _, row in dept_dict.iterrows():
    dept = row["department"]
    account = row["account"]
    number = row["number"]

    if dept not in nested_dict:
        nested_dict[dept] = {}  # Initialize dictionary for the department

    nested_dict[dept][account] = number


In [2]:
dept_dict.head()

,department,account,number
0,PE,Dance,2386
1,PE,PE,2518
2,PE,"conditioning, PE,dance",2383
3,Math,Math,2485
4,Math,"AP Calc,Math",5096


In [4]:
nested_dict["Math"]["AP Calc"]

5096